In [34]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [35]:
# Memuat data dari file JSON
with open('data_dana_desa.json') as f:
    data = json.load(f)

# Membuat DataFrame dari data
df = pd.DataFrame(data)
df.head()

,id,tahun,pendapatan,pembiayaan,belanja,bd_pemerintahan_desa,bd_pembangunan_desa,bd_pembinaan_kemasyarakatan,bd_pemberdayaan_masyarakat,bd_penanggulangan_bencana
0,1,2000,1500000000,10000000,1510000000,250000000,400000000,120000000,90000000,80000000
1,2,2001,1600000000,12000000,1612000000,260000000,420000000,130000000,95000000,90000000
2,3,2002,1700000000,14000000,1714000000,270000000,440000000,140000000,100000000,95000000
3,4,2003,1800000000,16000000,1816000000,280000000,460000000,150000000,110000000,100000000
4,5,2004,1900000000,18000000,1918000000,290000000,480000000,160000000,120000000,110000000


In [36]:
# Pilih kolom yang ingin diprediksi
columns_to_forecast = ['belanja', 'bd_pemerintahan_desa', 'bd_pembangunan_desa', 
                        'bd_pembinaan_kemasyarakatan', 'bd_pemberdayaan_masyarakat', 
                        'bd_penanggulangan_bencana']

# Normalisasi data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df[columns_to_forecast].values)

In [37]:
# Membagi data menjadi dataset pelatihan
x_train, y_train = [], []
for i in range(1, len(scaled_data)):
    x_train.append(scaled_data[i-1])
    y_train.append(scaled_data[i])

In [38]:
# Mengonversi ke numpy array
x_train, y_train = np.array(x_train), np.array(y_train)

# Reshape data untuk LSTM
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [39]:
# Membangun model LSTM
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=len(columns_to_forecast)))  # Output layer sesuai dengan kolom yang diprediksi

d:\project_python\HackAI\forecasting\myenv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [40]:
# Mengompilasi model
model.compile(optimizer='adam', loss='mean_squared_error')

# Melatih model
history = model.fit(x_train, y_train, epochs=100, batch_size=32, verbose=1)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - loss: 0.3326
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 0.3216
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 0.3100
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 0.2982
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step - loss: 0.2890
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - loss: 0.2689
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 0.2545
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 0.2426
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - loss: 0.2215
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - loss: 0.2140
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 0.1965
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 0.1740
Epoch 13/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 0.1537
Epoch 14/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 0.1389
Epoch 15/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - loss: 0.1020
Epoch 16/100
1/1 ━━

In [41]:
# Evaluasi model
y_pred = model.predict(x_train)  # Memprediksi data pelatihan
y_pred_inverse = scaler.inverse_transform(y_pred)  # Mengembalikan skala asli
y_train_inverse = scaler.inverse_transform(y_train)  # Mengembalikan skala asli

# Menghitung metrik evaluasi
mae = mean_absolute_error(y_train_inverse, y_pred_inverse)
mse = mean_squared_error(y_train_inverse, y_pred_inverse)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 742ms/step
Mean Absolute Error (MAE): 12185559.478260865
Mean Squared Error (MSE): 548561156552644.44


In [42]:
# Forecasting
years_to_forecast = 1  # Gantilah jika ingin memprediksi lebih dari satu tahun
predictions = []
last_data = scaled_data[-1].reshape(1, len(columns_to_forecast), 1)  # Data terakhir dalam bentuk yang benar

# Melakukan prediksi untuk tahun ke depan
for _ in range(years_to_forecast):
    prediction = model.predict(last_data)
    predictions.append(prediction)
    last_data = np.append(last_data[:, 1:, :], prediction.reshape(1, len(columns_to_forecast), 1), axis=1)

# Mengubah prediksi kembali ke skala asli
predicted_values = scaler.inverse_transform(np.array(predictions).reshape(-1, len(columns_to_forecast)))

print("Predicted values for the next year:")
print(predicted_values)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 658ms/step
Predicted values for the next year:
[[3.9231759e+09 4.8731894e+08 8.9663731e+08 3.5953773e+08 3.3004835e+08
  3.1457184e+08]]


In [43]:
model.save('lstm_model.h5')

In [28]:
import matplotlib.pyplot as plt

# Ambil data asli dari DataFrame atau array yang telah digunakan
# Asumsikan 'data' adalah DataFrame yang memuat data asli

# Mengambil data asli untuk kolom yang ingin divisualisasikan
actual_data = scaler.inverse_transform(scaled_data)

# Mengambil tahun untuk prediksi
last_year = years[-1] + 1  # Tahun setelah tahun terakhir
predicted_years = [last_year]  # Tahun prediksi

# Menggabungkan tahun dan nilai prediksi
for i in range(1, years_to_forecast + 1):
    predicted_years.append(last_year + i)

# Menggabungkan data asli dan prediksi
combined_years = list(years) + predicted_years
combined_values = np.vstack((actual_data, predicted_values))

# Plotting
plt.figure(figsize=(12, 6))
for i, column in enumerate(columns_to_forecast):
    plt.subplot(len(columns_to_forecast), 1, i + 1)
    plt.plot(years, actual_data[:, i], label='Actual', marker='o')
    plt.plot(predicted_years, predicted_values[:, i], label='Predicted', marker='x', linestyle='--')
    plt.title(f'Forecasting {column}')
    plt.xlabel('Year')
    plt.ylabel(column)
    plt.axvline(x=years[-1], color='r', linestyle='--', label='Forecast Start')
    plt.legend()

plt.tight_layout()
plt.show()


NameError: name 'years' is not defined

In [4]:
# Fungsi untuk melatih model LSTM dan melakukan prediksi
def forecast(data, column_name, years_to_forecast=1):
    # Normalisasi data
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data[[column_name]])

    # Memisahkan data menjadi train
    train_data = scaled_data[:-1]  # Data hingga tahun terakhir
    x_train = []
    y_train = []
    
    # Membuat dataset untuk pelatihan
    for i in range(1, len(train_data)):
        x_train.append(train_data[i-1])
        y_train.append(train_data[i])
    
    x_train, y_train = np.array(x_train), np.array(y_train)

    # Reshape data untuk LSTM
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

    # Membangun model LSTM
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))  # Output layer

    # Mengompilasi model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Melatih model
    model.fit(x_train, y_train, epochs=100, batch_size=32)

    # Menyimpan model
    model.save('lstm_model.h5')

    # Menyimpan evaluasi model
    y_pred = model.predict(x_train)  # Menggunakan data training untuk evaluasi
    y_pred_inverse = scaler.inverse_transform(y_pred)
    y_train_inverse = scaler.inverse_transform(y_train.reshape(-1, 1))
    
    # Menghitung evaluasi MAE dan MSE
    mae = mean_absolute_error(y_train_inverse, y_pred_inverse)
    mse = mean_squared_error(y_train_inverse, y_pred_inverse)
    
    # Menggunakan model untuk prediksi
    last_data = scaled_data[-1].reshape(1, 1, 1)  # Data tahun terakhir
    predictions = []

    # Forecasting untuk beberapa tahun ke depan
    for _ in range(years_to_forecast):
        prediction = model.predict(last_data)
        predictions.append(prediction)
        last_data = np.append(last_data[:, 1:, :], prediction.reshape(1, 1, 1), axis=1)

    # Mengubah prediksi kembali ke skala asli
    predicted_value = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))

    return predicted_value[0][0], mae, mse


In [5]:
# Memuat data dari file JSON
with open('data_dana_desa.json') as f:
    data = json.load(f)

# Membuat DataFrame dari data
df = pd.DataFrame(data)

# Melakukan forecasting untuk setiap rincian belanja
tahun_prediksi = 2024
forecast_results = {}

# Daftar rincian belanja
rincian_belanja = [
    'belanja',
    'bd. pemerintahan desa',
    'bd. pembangunan desa',
    'bd. pembinaan kemasyarakatan',
    'bd. pemberdayaan masyarakat',
    'bd. penanggulangan bencana'
]

# Melakukan prediksi untuk setiap rincian
for column in rincian_belanja:
    predicted_value, mae, mse = forecast(df, column)
    forecast_results[column] = predicted_value
    print(f"Prediksi {column} untuk tahun {tahun_prediksi}: {predicted_value:,.0f}")
    print(f"Evaluasi model untuk {column} - MAE: {mae:.2f}, MSE: {mse:.2f}")

Epoch 1/100


d:\project_python\HackAI\forecasting\myenv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.3260
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.3206
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.3146
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.3127
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.3063
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.3027
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.2971
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.2893
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.2846
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.2801
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.2778
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.2707
Epoch 13/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.2659
Epoch 14/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.2596
Epoch 15/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.2574
Epoch 16/100
1/1 ━━━━━━━━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 292ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 308ms/step
Prediksi belanja untuk tahun 2024: 2,730,360,064
Evaluasi model untuk belanja - MAE: 144322697.85, MSE: 28828761053454336.00


KeyError: "None of [Index(['bd. pemerintahan desa'], dtype='object')] are in the [columns]"

In [ ]:
# Melakukan prediksi untuk setiap rincian dan menyimpan hasilnya
for column in rincian_belanja:
    predicted_value, mae, mse, y_train_inverse, y_pred_inverse = forecast(df, column)
    forecast_results[column] = predicted_value

    # Visualisasi hasil prediksi dan data asli
    plt.figure(figsize=(12, 6))
    plt.plot(y_train_inverse, label='Data Asli')
    plt.plot(y_pred_inverse, label='Prediksi', color='orange')
    plt.title(f'Prediksi {column}')
    plt.xlabel('Tahun')
    plt.ylabel('Nilai')
    plt.axvline(x=len(y_train_inverse), color='r', linestyle='--', label='Tahun Prediksi')
    plt.legend()
    plt.show()
